## Imports...

In [2]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math 
from collections import defaultdict
from math import atan2, degrees, sqrt

### First load all of the static image gestures. These are gestures that have no movement.

In [3]:
gesture_path = './gestures'
# If you recieve nothing from these lines make sure you have run nthe data_in scripts.
gesture_files = glob.glob("./gestures/*.csv")

def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    return text


def remove_suffix(text, suffix):
    if text.endswith(suffix):
        return text[: -1 * len(suffix)]
    return text


static_gestures = {}
# iterate through all of the gestures... Static pose is finding the median point of the dataframe.

# TODO: refit the transcription process to lead from correct pose point to pose point. instead of 0-20, 
# we need to do the math to the connecting limb to the center of the heart.
for indx, gesture_file in enumerate(gesture_files):
  
  df = pd.read_csv(gesture_file)
  
  gesture_file = remove_prefix(gesture_file, './gestures/')
  gesture_file = remove_suffix(gesture_file, '.csv')
  
  file_parts = gesture_file.split('_')
  gesture_name = file_parts[0].lower()
  gesture_part = f"{file_parts[2]}".lower()
  if gesture_name == 'one':
    print(gesture_part)

  if df.empty:
      continue
  
  # convert string to time.
  df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S.%f').dt.time
  
  sdf = np.sort(df['time'].to_numpy(), axis=0)
  # Get the lower level bound median
  ts_median = sdf[math.floor(len(sdf) / 2 )]
  
  # Get the static point image/gesture...
  ndf = df[df['time'].isin([ts_median])]
  newGestureData = static_gestures.get(gesture_name, {})
  newGestureData[gesture_part] = ndf
  static_gestures[gesture_name] = newGestureData
  
print(static_gestures['one'])

right
left
pose
{'right':                 time  index         x         y             z
294  00:00:01.928231      0  0.421217  0.673906 -1.109492e-07
295  00:00:01.928231      1  0.403672  0.611394  1.546637e-03
296  00:00:01.928231      2  0.406900  0.548948  2.578361e-03
297  00:00:01.928231      3  0.421388  0.515959  4.016793e-03
298  00:00:01.928231      4  0.437194  0.514636  5.106071e-03
299  00:00:01.928231      5  0.426257  0.522519 -9.479048e-03
300  00:00:01.928231      6  0.433733  0.448504 -1.096984e-02
301  00:00:01.928231      7  0.438970  0.408821 -9.899020e-03
302  00:00:01.928231      8  0.443120  0.377513 -9.628103e-03
303  00:00:01.928231      9  0.449079  0.535689 -1.012208e-02
304  00:00:01.928231     10  0.448800  0.486791 -1.127748e-02
305  00:00:01.928231     11  0.439842  0.502041 -6.422864e-03
306  00:00:01.928231     12  0.433441  0.523711 -2.302012e-03
307  00:00:01.928231     13  0.465129  0.555540 -9.614064e-03
308  00:00:01.928231     14  0.456896  0.521

In [20]:
limb_connections = [
    [0, 1, 2, 3, 4],
    [0, 5, 6, 7, 8],
    [0, 9, 10, 11, 12],
    [0, 13, 14, 15, 16],
    [0, 17, 18, 19, 20],
    [2, 5, 9, 13, 17],
]

coordinates = static_gestures['one']['right'].to_numpy()[:, 2:]
connections = np.array(limb_connections)

def angle_between_360(angle):
    """ Get the angle between 0 and 360."""
    return angle if angle >= 0 else 360 + angle


def get_angles(coordinates: np.array, connections: np.array) -> np.array:
    """ "
    Get the angles of each vector formed by the coordinates and the connections.

    Inputs:
        coordinates: np.array
            A 2D array containing the coordinates of the points.
        connections: np.array
            A 2D array containing the connections between the points.
            Each row is a different connection.

    Outputs:
        angles: np.array
            A 3D array containing the angles between the vectors.
            The size of the array is (connections.shape[0], connections.shape[1]-1,3).
    """
    angles = np.zeros((connections.shape[0], connections.shape[1] - 1, 3))
    for i in range(connections.shape[0]):
        for j in range(connections.shape[1] - 1):
            v = [
                coordinates[connections[i, j + 1], 0] - coordinates[connections[i, j], 0],
                coordinates[connections[i, j + 1], 1] - coordinates[connections[i, j], 1],
                coordinates[connections[i, j + 1], 2] - coordinates[connections[i, j], 2],
            ]
            angles[i, j, 0] = angle_between_360(degrees(atan2(v[1], v[2])))
            angles[i, j, 1] = angle_between_360(degrees(atan2(-v[0], sqrt(v[1] ** 2 + v[2] ** 2))))
            angles[i, j, 2] = angle_between_360(degrees(atan2(v[0], -v[1])))
    return angles

angles = get_angles(coordinates, connections)

In [9]:
# Loop through our images to a static pose we know works.

all_gestures = static_gestures.keys()
  
# Proof of concept for the hand gesture "1"
test_pose_name = 'one'
static_base_gesture = static_gestures['one']['right'].to_numpy()

for pose_name in all_gestures:
  # print(pose_name)
  pose = static_gestures[pose_name]
  if pose.get('right', False) is False:
    continue
  pose_points = pose['right'].to_numpy()
  
  is_a_match = True
  
  for idx, pose_point in enumerate(pose_points):
    
    x_angle = pose_point[2]
    y_angle = pose_point[3]
    z_angle = pose_point[4]
    
    base_x = static_base_gesture[idx][2]
    base_y = static_base_gesture[idx][3]
    base_z = static_base_gesture[idx][4]
    
    # Compute the error
    x_error = abs(base_x - x_angle)
    y_error = abs(base_y - y_angle)
    z_error = abs(base_z - z_angle)
    
    if(x_error > .1) or (y_error > .1) or (z_error > .1):
      is_a_match = False
  if is_a_match:
    print(f"The Pose {pose_name} matches the pose {test_pose_name} right in the middle of the video")
    # Within a threshold, say the pose is correct or not...
    # TODO: Add more test cases here, maybe add your own pictures in.....
    
  # This is the hard part... We need to fine tune the model for every static pose..
  # TODO: fine tune our error function to work for only the pose we want it to and no other poses.


The Pose one matches the pose one right in the middle of the video
The Pose change matches the pose one right in the middle of the video
The Pose complex matches the pose one right in the middle of the video
The Pose brave matches the pose one right in the middle of the video
The Pose boring matches the pose one right in the middle of the video
The Pose alone matches the pose one right in the middle of the video
